In [ ]:
'''
Se não conseguir importar um arquivo manualmente rode esse bloco
'''

from google.colab import files
files.upload()

In [ ]:
!unzip -q model.zip

To ensure reproducibility while running on google colab make sure the runtime is new and no network has been trained during the runtime.

#**Core**

In [ ]:
import os
os.environ['DDE_BACKEND'] = 'tensorflow'
# os.environ['DDE_BACKEND'] = 'pytorch'

from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib as mpl
import json
import re
import random


import tensorflow as tf
import numpy as np

from scipy.integrate import quad, dblquad
from math import sqrt


!pip install deepxde -q
import deepxde as dde
import keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.4/184.4 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 5.8 MB/s eta 0:00:00


Using backend: tensorflow
Other supported backends: tensorflow.compat.v1, pytorch, jax, paddle.
paddle supports more examples now and is recommended.
Enable just-in-time compilation with XLA.



In [ ]:
'''
Setting json encoder for converting numpy.integer to python's int when saving model data.
'''
class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        # if isinstance(onj, np.ndarray):
        #   return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)


In [ ]:
# setting seed to a fixed value
def set_seed(seed_value = 0):
  np.random.seed(seed_value)
  tf.random.set_seed(seed_value)
  random.seed(seed_value)
  dde.config.set_random_seed(seed_value)

set_seed()

In [ ]:
class PDE:
  def __init__(self, p, net_data):
    self.p = p
    self.xmin, self.xmax = [-10, 1], [10, 2]
    self.tmax = 15
    self.net_data = net_data

    self.lr=1e-3
    self.resample = True
    self.resample_period = 10
    self.borda = "exact"
    self.vl2relerror_xt = np.vectorize(self.l2relerror_xt)
    self.vdde_l2relerror = np.vectorize(self.dde_l2relerror)
    self.eps = 1e-5
    self.error_data = {"eps" : self.eps}

    self.test_xmin, self.test_xmax = self.xmin, self.xmax
    self.test_tmin, self.test_tmax = 0, self.tmax


  def set_data(self):
    self.geom = dde.geometry.geometry_2d.Rectangle(xmin=self.xmin, xmax=self.xmax)
    self.timeDomain = dde.geometry.TimeDomain(0, self.tmax)
    self.geomtime = dde.geometry.GeometryXTime(self.geom, self.timeDomain)

    self.ic = dde.icbc.IC(self.geomtime, self.pde_sol, lambda _, on_initial: on_initial)
    if self.borda == 0:
      self.bc = dde.icbc.DirichletBC(self.geomtime, lambda x: 0, self.on_boundary_X, component=0)
    elif self.borda == "exact":
      self.bc = dde.icbc.DirichletBC(self.geomtime, self.pde_sol, self.on_boundary_X)
    if self.borda != "off":
      self.data = dde.data.TimePDE(
                          self.geomtime,
                          self.pde,
                          [self.bc, self.ic],
                          num_domain = self.net_data["ND"],
                          num_boundary = self.net_data["Nb"],
                          num_initial = self.net_data["N0"]
                          )
    else:
      self.data = dde.data.TimePDE(
                          self.geomtime,
                          self.pde,
                          [self.ic],
                          num_domain = self.net_data["ND"],
                          num_boundary = self.net_data["Nb"],
                          num_initial = self.net_data["N0"]
                          )

    self.net = dde.nn.FNN([3] + [self.net_data["neurons"]] * self.net_data["layers"] + [1], "tanh", "zeros")
    for d in self.net.denses:
        d.kernel_initializer = tf.keras.initializers.GlorotNormal()
    self.model = dde.Model(self.data, self.net)

    # domínio de previsão e cálculo de erro
    self.x_num = int(100 + 5*(self.test_xmax[0] - self.test_xmin[0]))
    self.x = np.linspace(self.test_xmin[0], self.test_xmax[0], self.x_num)

  def on_boundary_X(self, x, on_boundary):
    return on_boundary and (dde.utils.isclose(x[0], self.xmin[0]) or dde.utils.isclose(x[0], self.xmax[0]))



  def pde(self, x, y):
    dy_t = dde.grad.jacobian(y, x, i=0, j=2)
    dy_x = dde.grad.jacobian(y, x, i=0, j=0)
    dy_xxx = dde.grad.hessian(dy_x, x, i=0, j=0)
    pde = dy_t + y**self.p * dy_x + dy_xxx
    return pde


  def pde_sol(self, X):
    x_arr = X[:, 0:1]
    a_arr = X[:, 1:2]
    t_arr = X[:, 2:3]


    if self.p == 1:
      c = a_arr / 3
      k = np.sqrt(a_arr / 12)
      return a_arr*np.power(np.cosh(k*(x_arr - c*t_arr)), -2)

    if self.p == 2:
      c = a_arr**2 / 6
      k = a_arr / sqrt(6)
      return a_arr*np.power(np.cosh(k*(x_arr - c*t_arr)), -1)


  def train(self, iterations):
    try:
      os.makedirs(f"model/cpkt")
    except FileExistsError:
      pass

    display_every = 1_000
    checkpointer = dde.callbacks.ModelCheckpoint(
      f"model/ckpt/",
      period=1000,
      verbose=1,
      save_better_only=True
    )
    if self.resample:
      resampler = dde.callbacks.PDEPointResampler(period=self.resample_period)
      return self.model.train(iterations=iterations,
                        display_every=display_every,
                        callbacks=[resampler])


    return self.model.train(iterations=iterations,
                            display_every=display_every,
                            #model_save_path=f"model/ckpt/",
                            #callbacks=[checkpointer]
                            )

  def get_arr_X(self, t, a):
    t_array = np.full_like(self.x, t)
    a_array = np.full_like(self.x, a)
    X = np.stack((self.x, a_array, t_array), axis=1)
    return X

  def dde_l2relerror(self, t, a):
    X = self.get_arr_X(t, a)
    y_pred = self.model.predict(X)   #.reshape(self.x_num)
    y_true = self.pde_sol(X)
    return dde.metrics.l2_relative_error(y_true, y_pred)

  def l2relerror_xt(self, a):
    func = lambda x, t: (self.pde_sol(np.array([[x, a, t]])) - self.model.predict(np.array([[x, a, t]])))**2
    l2_norm_diff = sqrt(dblquad(func, self.xmin[0], self.xmax[0], lambda t: 0, lambda t: self.tmax, epsabs=self.eps, epsrel=self.eps)[0])
    l2_norm_true = sqrt(dblquad(lambda x, t: self.pde_sol(np.array([[x, a, t]]))**2, self.xmin[0], self.xmax[0], lambda t: 0, lambda t: self.tmax, epsabs=self.eps, epsrel=self.eps)[0])
    return l2_norm_diff / l2_norm_true


  def gif(self, a):
    Figure = plt.figure()
    predicted_line = plt.plot([], 'b-')[0]
    solution_line = plt.plot([], 'r--')[0]
    plt.xlim(self.test_xmin[0], self.test_xmax[0])
    plt.ylim(-0.5, 2.5)
    frames=120
    def AnimationFunction(frame):
        K= (self.test_tmax - self.test_tmin)/frames
        t = self.test_tmin + K*frame
        plt.title(f"t={t:.2f}")
        X = self.get_arr_X(t=t, a=a)
        y_pred = self.model.predict(X)
        y_sol = self.pde_sol(X)
        predicted_line.set_data((self.x, y_pred))
        solution_line.set_data((self.x, y_sol))

    anim_created = FuncAnimation(Figure, AnimationFunction, frames=frames, interval=25)
    writer = animation.PillowWriter(fps=24,
                                    metadata=dict(artist='Me'),
                                    bitrate=1800)
    anim_created.save(f'model/prediction__a{a}__xrange{self.test_xmin[0]}_{self.test_xmax[0]}__trange{self.test_tmin}_{self.test_tmax}.gif', writer=writer)
    plt.close()

  def dde_l2relerror_scatter(self, grid_size=100):
    fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
    # fig.set_figheight(1.9685)
    # fig.set_figwidth(5.1181)
    # fig.set_dpi(96)
    ax.cla()
    ax.set_xlim(self.test_xmin[1], self.test_xmax[1])
    ax.set_ylim(self.test_tmin, self.test_tmax)
    ax.set_xlabel("a")
    ax.set_ylabel("t")
    ax.set_zlabel("Error")
    plt.title("L2 Relative Error")

    A = np.linspace(self.test_xmin[1], self.test_xmax[1], grid_size)
    T = np.linspace(self.test_tmin, self.test_tmax, grid_size)
    A, T = np.meshgrid(A, T)
    Z = self.vdde_l2relerror(t=T, a=A)
    dde_l2relerror_max = np.max(Z)
    dde_l2relerror_mean = np.mean(Z)
    self.error_data["dde_l2relerror_max"] = dde_l2relerror_max
    self.error_data["dde_l2relerror_mean"] = dde_l2relerror_mean

    # ax.scatter(A, T, Z, s = 5, marker = 'o', cmap=cm.inferno, c=Z)
    surf = ax.plot_surface(A, T, Z, cmap=cm.inferno, linewidth=0, antialiased=True)
    #norm=mpl.colors.Normalize(0, np.max(Z))
    #np.linspace(0,np.max(Z),100)
    fig.colorbar(surf, shrink=0.5, aspect=15, ticks=np.linspace(0,np.max(Z),10), pad=0.1, boundaries=np.linspace(0,np.max(Z),100))

    plt.savefig(f"model/dde_l2relerror_x__xrange{self.test_xmin[0]}_{self.test_xmax[0]}__trange{self.test_tmin}_{self.test_tmax}.png")


  def l2relerror_xt_scatter(self):
    '''
    L2 Norm integrating in x and t.
    '''
    plt.clf()
    fig, ax = plt.subplots()
    # fig.set_figheight(1.9685)
    # fig.set_figwidth(5.1181)
    # fig.set_dpi(96)
    a = np.linspace(self.xmin[1], self.xmax[1], 50)
    error = self.vl2relerror_xt(a)
    self.error_data["l2relerror_xt_max"] = np.max(error)
    self.error_data["l2relerror_xt_mean"] = np.mean(error)

    ax.plot(a, error)
    ax.set_xlabel("a")
    ax.set_ylabel("Error")
    plt.title("l2relerror_xt")
    plt.savefig(f"model/l2relerror_xt.png")


  def restore_model(self, filename):
    del self.model

    self.model = dde.Model(self.data, self.net)
    self.model.restore(filename)
    self.model.compile("adam", lr=self.lr)
    self.model.predict(self.get_arr_X(a=self.xmax[1], t=self.tmax))
    self.model.restore(filename)
    self.model.compile("adam", lr=self.lr)

  def get_error_array(self):
    '''
    Returns an array whose length is the number of checkpoints saved in the ckpt folder
    With columns [max_err, mean_err, ckpt]
    '''
    A = np.linspace(self.test_xmin[1], self.test_xmax[1], 20)
    T = np.linspace(self.test_tmin, self.test_tmax, 20)
    A, T = np.meshgrid(A, T)

    ckptlist = set()
    for file in os.listdir("model/ckpt/"):
      isckpt = re.search("\d+\.ckpt", file)
      if isckpt is not None:
        ckptlist.add(f"{isckpt.group(0).replace('.ckpt', '')}")

    ckptlist = list(ckptlist)
    ckptlist = [int(ckpt) for ckpt in ckptlist]
    ckptlist.sort() # remover essa linha depois

    error_array = np.zeros((len(ckptlist), 3))
    for i, ckpt in enumerate(ckptlist):
      self.load_ckpt(f"model/ckpt/-{ckpt}.ckpt")
      max_err = np.max(self.vdde_l2relerror(t=T, a=A))
      mean_err = np.mean(self.vdde_l2relerror(t=T, a=A))
      error_array[i, 0] = max_err
      error_array[i, 1] = mean_err
      error_array[i, 2] = ckpt

    self.error_array = error_array
    return error_array


  def get_error_plot(self):#, start, end):
    '''
    Plots error by iteration
    '''
    fig, ax = plt.subplots()
    ax.cla()
    ax.set_xlabel("Iteration")
    ax.set_ylabel("l2relerror")
    ax.set_yscale("log")
    max_err = self.error_array[:, 0]
    mean_err = self.error_array[:, 1]
    ckpts = self.error_array[:, 2]
    ax.scatter(ckpts, max_err, label="l2relerror_max")
    ax.scatter(ckpts, mean_err, label="l2relerror_mean")
    ax.legend()
    plt.savefig(f"model/l2relerror_iterations.png")


  def save_data(self):
    self.net_data["lr"] = self.lr
    self.net_data["resample"] = self.resample
    self.net_data["resample_period"] = self.resample_period
    self.net_data["xmin"] = self.xmin
    self.net_data["xmax"] = self.xmax
    self.net_data["tmax"] = self.tmax
    self.net_data["borda"] = self.borda
    self.error_data["test_xmin"] = self.test_xmin
    self.error_data["test_xmax"] = self.test_xmax
    self.error_data["test_tmin"] = self.test_tmin
    self.error_data["test_tmax"] = self.test_tmax


    try:
      os.makedirs(f"model/")
    except FileExistsError:
      pass

    try:
      with open(f"model/net_data.json", "x") as outfile:
        json.dump(self.net_data, outfile, indent=2)
    except FileExistsError:
      with open(f"model/net_data.json", "w") as outfile:
        json.dump(self.net_data, outfile, indent=2)

    try:
      with open(f"model/error_data.json", "x") as outfile:
        json.dump(self.error_data, outfile, indent=2)
    except FileExistsError:
      with open(f"model/error_data.json", "w") as outfile:
        json.dump(self.error_data, outfile, indent=2)


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# **KdV**

In [ ]:
set_seed()
net_data = {
    "neurons" : 60,
    "layers" : 5,
    "ND" :10_000,
    "Nb" : 4_000,
    "N0" : 4_000,
}
kdv = PDE(p=1, net_data=net_data)

In [ ]:
kdv.xmin, kdv.xmax = [-10, 1], [10, 2]
kdv.tmax = 15
kdv.borda = "exact"
kdv.set_data()

In [ ]:
set_seed()

kdv.resample = True
kdv.resample_period = 1000

kdv.model.compile("adam", lr=1e-2)
losshistory, train_state = kdv.train(iterations=3_000)

kdv.model.compile("adam", lr=5e-3)
losshistory, train_state = kdv.train(iterations=3_000)

kdv.model.compile("adam", lr=1e-3)
losshistory, train_state = kdv.train(iterations=4_000)

kdv.model.compile("adam", lr=7e-4)
losshistory, train_state = kdv.train(iterations=5_000)

kdv.model.compile("adam", lr=5e-4)
losshistory, train_state = kdv.train(iterations=5_000)

kdv.model.compile("adam", lr=3e-4)
losshistory, train_state = kdv.train(iterations=5_000)

kdv.model.compile("adam", lr=1e-4)
losshistory, train_state = kdv.train(iterations=15_000)

kdv.model.compile("adam", lr=7e-5)
losshistory, train_state = kdv.train(iterations=15_000)

kdv.model.compile("adam", lr=5e-5)
losshistory, train_state = kdv.train(iterations=15_000)

kdv.model.compile("adam", lr=3e-5)
losshistory, train_state = kdv.train(iterations=15_000)

kdv.model.compile("adam", lr=1e-5)
losshistory, train_state = kdv.train(iterations=15_000)

dde.saveplot(losshistory, train_state, issave=True, isplot=True)

Compiling model...
'compile' took 0.032927 s

Training model...

Step      Train loss                        Test loss                         Test metric
0         [3.59e-03, 1.18e-01, 4.15e-01]    [3.59e-03, 1.18e-01, 4.15e-01]    []  
1         [4.47e-01, 8.30e+00, 8.35e+00]    [4.47e-01, 8.30e+00, 8.35e+00]    []  
2         [1.65e-03, 2.83e-01, 1.54e-01]    [1.65e-03, 2.83e-01, 1.54e-01]    []  
3         [1.98e-02, 1.05e+00, 1.50e+00]    [1.98e-02, 1.05e+00, 1.50e+00]    []  
4         [2.92e-02, 4.18e-01, 3.74e-01]    [2.92e-02, 4.18e-01, 3.74e-01]    []  
5         [1.68e-02, 4.07e-01, 6.26e-01]    [1.68e-02, 4.07e-01, 6.26e-01]    []  
6         [1.34e-02, 4.12e-01, 7.58e-01]    [1.34e-02, 4.12e-01, 7.58e-01]    []  
7         [2.31e-02, 4.26e-01, 2.39e-01]    [2.31e-02, 4.26e-01, 2.39e-01]    []  
8         [3.48e-02, 5.21e-01, 1.38e-01]    [3.48e-02, 5.21e-01, 1.38e-01]    []  
9         [2.09e-02, 3.10e-01, 2.32e-01]    [2.09e-02, 3.10e-01, 2.32e-01]    []  
10        [1.57

KeyboardInterrupt: 

In [ ]:
kdv.model.save("model/ckpt")

In [ ]:
kdv.restore_model("ckpt-100000.weights.h5")

In [ ]:
kdv.dde_l2relerror_scatter(grid_size=100)
# kdv.gif(a=2)

# kdv.save_data()

In [ ]:
for key, value in (kdv.net_data | kdv.error_data).items():
  print(f"{key}: {value}")

In [ ]:
!zip -r model.zip model -q

In [ ]:
from google.colab import files
files.download("model.zip")

#**mKdV**

In [ ]:
set_seed()
net_data = {
    "neurons" : 60,
    "layers" : 5,
    "ND" :10_000,
    "Nb" : 4_000,
    "N0" : 4_000,
}
mkdv = PDE(p=2, net_data=net_data)

In [ ]:
mkdv.xmin, mkdv.xmax = [-10, 1], [10, 2]
mkdv.tmax = 15
mkdv.borda = "exact"
mkdv.set_data()

In [ ]:
mkdv.restore_model("ckpt-100000.weights.h5")

Compiling model...
'compile' took 0.094834 s

Compiling model...
'compile' took 0.005641 s



In [ ]:
def TRAINLR(iterations, initial_learning_rate, final_learning_rate):

  lr_schedule = keras.optimizers.schedules.ExponentialDecay(
      initial_learning_rate,
      decay_steps=iterations,
      decay_rate=final_learning_rate/initial_learning_rate,
      staircase=False)

  mkdv.model.compile(keras.optimizers.Adam(learning_rate=lr_schedule))

  mkdv.train(iterations=iterations)


In [ ]:
set_seed()

mkdv.resample = True
mkdv.resample_period = 1000

mkdv.model.compile("adam", lr=1e-2)
losshistory, train_state = mkdv.train(iterations=4_000)

mkdv.model.compile("adam", lr=5e-3)
losshistory, train_state = mkdv.train(iterations=2_000)

mkdv.model.compile("adam", lr=1e-3)
losshistory, train_state = mkdv.train(iterations=4_000)

mkdv.model.compile("adam", lr=7e-4)
losshistory, train_state = mkdv.train(iterations=5_000)

mkdv.model.compile("adam", lr=5e-4)
losshistory, train_state = mkdv.train(iterations=5_000)

mkdv.model.compile("adam", lr=3e-4)
losshistory, train_state = mkdv.train(iterations=7_000)

mkdv.model.compile("adam", lr=1e-4)
losshistory, train_state = mkdv.train(iterations=20_000)

mkdv.model.compile("adam", lr=7e-5)
losshistory, train_state = mkdv.train(iterations=15_000)

mkdv.model.compile("adam", lr=5e-5)
losshistory, train_state = mkdv.train(iterations=15_000)

mkdv.model.compile("adam", lr=3e-5)
losshistory, train_state = mkdv.train(iterations=8_000)

mkdv.model.compile("adam", lr=1e-5)
losshistory, train_state = mkdv.train(iterations=15_000)


dde.saveplot(losshistory, train_state, issave=True, isplot=True)

Compiling model...
'compile' took 0.000717 s

Training model...

Step      Train loss                        Test loss                         Test metric
0         [3.45e-03, 1.05e-01, 3.71e-01]    [3.45e-03, 1.05e-01, 3.71e-01]    []  


KeyboardInterrupt: 

In [ ]:
mkdv.model.save("model/ckpt")

'model/ckpt-100000.weights.h5'

In [ ]:
print(dde.model.LossHistory)

In [ ]:
grid_size=20

In [ ]:
A= np.linspace(1,2,50)
T= np.linspace(0,15,50)

In [ ]:
np.linspace(0,10,10)

In [ ]:
V = mkdv.vdde_l2relerror(a=2, t=T)

In [ ]:
max(V)

0.008360439958863751

In [ ]:
try:
    os.makedirs(f"model/")
except FileExistsError:
    pass

In [ ]:
set_seed()
mkdv.resample = True
mkdv.resample_period = 100
mkdv.model.compile("adam", lr=1e-3)
mkdv.train(iterations=10000)

Compiling model...
'compile' took 0.008921 s

Training model...

0         [3.45e-03, 1.05e-01, 3.71e-01]    [3.45e-03, 1.05e-01, 3.71e-01]    []  
1000      [1.13e-03, 8.76e-04, 1.37e-04]    [1.12e-03, 8.76e-04, 1.37e-04]    []  
2000      [5.37e-04, 1.37e-04, 3.85e-05]    [5.60e-04, 1.37e-04, 3.85e-05]    []  
3000      [3.36e-04, 7.03e-05, 1.43e-05]    [3.69e-04, 7.03e-05, 1.43e-05]    []  
4000      [2.25e-04, 4.41e-05, 8.75e-06]    [2.56e-04, 4.41e-05, 8.75e-06]    []  
5000      [1.71e-04, 1.52e-04, 1.54e-04]    [1.75e-04, 1.52e-04, 1.54e-04]    []  
6000      [1.09e-04, 6.08e-05, 5.60e-05]    [1.22e-04, 6.08e-05, 5.60e-05]    []  
7000      [8.26e-05, 9.84e-05, 2.65e-05]    [8.55e-05, 9.84e-05, 2.65e-05]    []  
8000      [5.75e-05, 1.22e-05, 7.70e-06]    [6.25e-05, 1.22e-05, 7.70e-06]    []  
9000      [4.27e-05, 3.28e-06, 2.92e-06]    [4.48e-05, 3.28e-06, 2.92e-06]    []  
10000     [3.42e-05, 2.63e-05, 2.29e-05]    [3.51e-05, 2.63e-05, 2.29e-05]    []  

Best model at step 90

(<deepxde.model.LossHistory at 0x7e92a49c9330>,
 <deepxde.model.TrainState at 0x7e92a49c9540>)

In [ ]:
mkdv.dde_l2relerror_scatter(grid_size=50)
# mkdv.gif(a=2)
# #mkdv.l2relerror_xt_scatter() #slow to compute, don't use gpu
mkdv.save_data()

In [ ]:
mkdv.save_data()

In [ ]:
mkdv.gif(a=2)

In [ ]:
for key, value in (mkdv.net_data | mkdv.error_data).items():
  print(f"{key}: {value}")

neurons: 60
layers: 5
ND: 10000
Nb: 4000
N0: 4000
lr: 0.001
resample: True
resample_period: 1000
xmin: [-10, 1]
xmax: [10, 2]
tmax: 15
borda: exact
eps: 1e-05
dde_l2relerror_max: 0.0071376642909363845
dde_l2relerror_mean: 0.002095247049670496
test_xmin: [-10, 1]
test_xmax: [10, 2]
test_tmin: 0
test_tmax: 15


In [ ]:
!zip -r model.zip model -q

In [ ]:
from google.colab import files
files.download("model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
 # from shutil import rmtree
# rmtree("model")